[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay24.ipynb)

# --- Day 24: Lobby Layout ---

In [97]:
using System.IO;
using System.Numerics;
using System.Text.RegularExpressions;

In [98]:
enum Neighbors
{
    E, SE, SW, W, NW, NE
}

In [99]:
enum Colors
{
    White, Black
}

In [100]:
class Tile
{
    public Colors Color { get; set; }
    
    public void Flip()
    {
        Color = Color == Colors.White ? Colors.Black : Colors.White;
    }
}

Using the [Cube Coordinate](https://www.redblobgames.com/grids/hexagons/#coordinates-cube) for the hexagons.

In [130]:
readonly Dictionary<Neighbors, Vector3> directions = new()
{
    {Neighbors.E, new Vector3(+1, -1, 0)},
    {Neighbors.SE, new Vector3(0, -1, +1)},
    {Neighbors.SW, new Vector3(-1, 0, +1)},
    {Neighbors.W, new Vector3(-1, +1, 0)},
    {Neighbors.NW, new Vector3(0, +1, -1)},
    {Neighbors.NE, new Vector3(+1, 0, -1)},
};

In [131]:
Vector3 GetTilePosition(string steps) =>
    Regex.Matches(steps, @"e|se|sw|w|nw|ne")
        .Select(grp => Enum.Parse<Neighbors>(grp.Value.ToUpper()))
        .Select(n => directions[n])
        .Aggregate(new Vector3(0, 0, 0), (a, b) => a + b);

In [195]:
var tiles = File.ReadAllLines(@"input/24.txt")
    .Select(line => GetTilePosition(line));

In [196]:
Dictionary<Vector3, Tile> grid = new();
foreach (var tile in tiles)
{
    if (grid.ContainsKey(tile))
    {
        grid[tile].Flip();
    }
    else
    {
        grid[tile] = new Tile {Color = Colors.Black};
    }
}

In [197]:
grid.Values.Where(t => t.Color == Colors.Black).Count()

293

# --- Part Two ---

In [198]:
Dictionary<Vector3, Tile> FlipTiles(Dictionary<Vector3, Tile> grid)
{
    var newGrid = new Dictionary<Vector3, Tile>();
    
    var max = grid.Keys
        .Select(k => new float[] {Math.Abs(k.X), Math.Abs(k.Y), Math.Abs(k.Z)})
        .SelectMany(x => x)
        .Max() + 1;
    
    for (var x = -max; x <= max; x++)
    {
        for (var y = -max; y <= max; y++)
        {
            for (var z = -max; z <= max; z++)
            {
                if (x + y + z == 0)
                {
                    var key = new Vector3(x, y, z);
                    if (!grid.ContainsKey(key))
                    {
                        grid[key] = new Tile();
                    }
                }
            }
        }
    }
    
    foreach (var tile in grid)
    {
        var adj = directions
            .Select(d => grid.ContainsKey(tile.Key + d.Value) ? 
                    grid[tile.Key + d.Value].Color : Colors.White)
            .Where(c => c == Colors.Black)
            .Count();
        
        if (tile.Value.Color == Colors.Black)
        {
            newGrid[tile.Key] = adj == 0 || adj > 2 
                ? new Tile {Color = Colors.White}
                : new Tile {Color = Colors.Black};
        }
        else if (tile.Value.Color == Colors.White)
        {
            newGrid[tile.Key] = adj == 2 
                ? new Tile {Color = Colors.Black} 
                : new Tile {Color = Colors.White};
        }
    }
    
    return newGrid;
}

In [199]:
for (var i = 0; i < 100; i++)
{
    grid = FlipTiles(grid);
}

grid.Values.Where(t => t.Color == Colors.Black).Count()

3967